In [139]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import re
import pickle
import numpy as np


In [140]:
file_path = 'updated_health_insurance_policies.csv'  # Replace with your file path
data = pd.read_csv(file_path)


In [141]:
def extract_numeric(value):
    if isinstance(value, str):
        value = re.sub(r'[^\d.]', '', value)  # Remove non-numeric characters
        return float(value) if value else 0.0
    return value


In [142]:
fitness_columns = ['Fitness_Score1', 'Fitness_Score2', 'Fitness_Score3', 'Fitness Score_4']
data['Composite_Fitness_Score'] = data[fitness_columns].mean(axis=1)

In [143]:
processed_data = data[['Brand_Name','Policy_Name','Cashless_Hospitals', 'Coverage_Amount', 'Monthly_Premium',
                       'Annual_Premium', 'Claim_Settlement_Ratio', 'Composite_Fitness_Score','Salary']]


In [144]:
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(processed_data[['Composite_Fitness_Score','Salary']])

In [145]:
knn_model = NearestNeighbors(n_neighbors=12, algorithm='auto')
knn_model.fit(normalized_features)

NearestNeighbors(n_neighbors=12)

In [146]:
with open('knn_model.pkl', 'wb') as file:
    pickle.dump({
        'model': knn_model,
        'scaler': scaler,
        'data': processed_data
    }, file)
print("Model and preprocessing objects saved to knn_model.pkl")

Model and preprocessing objects saved to knn_model.pkl


In [147]:
with open('knn_model.pkl', 'rb') as file:
    saved_objects = pickle.load(file)

knn_model = saved_objects['model']
scaler = saved_objects['scaler']
processed_data = saved_objects['data']

print("Model and preprocessing objects loaded successfully.")


Model and preprocessing objects loaded successfully.


In [148]:
def recommend_policies(fitness_score, model, data, scaler,salary):
    # Normalize the input fitness score with dummy data for other features
    input_features = [fitness_score,salary]
    normalized_input = scaler.transform([input_features])[0]
    
    # Find the nearest neighbors
    distances, indices = model.kneighbors([normalized_input])
    
    # Get the recommended policies
    recommendations = data.iloc[indices[0]]
    return recommendations[['Brand_Name','Policy_Name','Cashless_Hospitals', 'Coverage_Amount', 'Monthly_Premium', 
                             'Annual_Premium', 'Claim_Settlement_Ratio']]

In [153]:
fitness_score = 70
salary=1000000
recommended_policies = recommend_policies(fitness_score, knn_model, processed_data, scaler,salary)

c:\Users\nayak\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [154]:
print("Recommended Policies:")
print(recommended_policies)

Recommended Policies:
              Brand_Name                                        Policy_Name  \
422            Niva Bupa                             Aspire Gold + (Direct)   
391      Universal Sompo                                        A Plus Gold   
597          IFFCO Tokio                        Individual Health Protector   
64        Royal Sundaram                                     Lifeline Elite   
545          Care Health          Care Advantage with Global Coverage Ex US   
728      Universal Sompo         A Plus Diamond + PED waiting period waiver   
262      Universal Sompo          Complete HealthCare Insurance - Essential   
355          Star Health                    Comprehensive with PED Buy Back   
526      Universal Sompo            Complete HealthCare Insurance - Premier   
326  New India Assurance                           New India Mediclaim 2012   
744        Manipal Cigna  Lifetime Health Global with World Wide Medical...   
502        ICICI Lombard  Heal